In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets as widgets
# Colors used
colors = ['#FF5851', '#3d9dfc', '#d852fa', '#494a49', '#212121' , '#ffffff', '#727372', '#03fc39']

In [ ]:
df=pd.read_csv("Historical Product Demand.csv")
df

<p style="color:purple;font-size:20px;">Data Exploration of the Data Set</p>

In [ ]:
df.head()

In [ ]:
df.tail()

<p style="color:MediumVioletred;font-size:20px;">Check Datatype</p>

In [ ]:
df.info()

<p style="color:MediumVioletRed;font-size:20px;">Check Statistics of data set</p>

In [ ]:
df.describe()

<p style="color:MediumVioletRed;font-size:20px;">Checking Missing Values</p>

In [ ]:
df.isna().sum()

<p style="color:MediumVioletRed;font-size:20px;">Check Duplicates</p>

In [ ]:
df.duplicated().sum()

<p style="color:MediumVioletRed;font-size:20px;">Checking the number of unique values of each column</p>

In [ ]:
df.nunique()

<p style="color:MediumVioletRed;font-size:20px;">Print numerical and categorical columns</p>

In [ ]:
numeric_columns = [column for column in df.columns if df[column].dtype != 'O']
categorical_columns = [column for column in df.columns if df[column].dtype == 'O']

# print columns
print('We have {} numerical columns(features) : {}'.format(len(numeric_columns), numeric_columns))
print('\nWe have {} categorical columns(features) : {}'.format(len(categorical_columns), categorical_columns))

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("Historical Product Demand.csv")

# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ensure 'Order_Demand' is a string before applying string operations
df['Order_Demand'] = df['Order_Demand'].astype(str)

# Format 'Order_Demand' to numeric
df['Order_Demand'] = df['Order_Demand'].str.replace('(', '-')
df['Order_Demand'] = df['Order_Demand'].str.replace(')', '')
df['Order_Demand'] = pd.to_numeric(df['Order_Demand'])

# Check the dataframe
print(df.head())


Negative Values

Order demands in this dataset include negative values. These can be either order adjustments or order returns.
Let's take a closer look into this by examining some products that have negative demand values.

In [ ]:
negative = df.loc[data['Order_Demand'] < 0]
negative_pv = pd.pivot_table(negative, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum
                            ).rename(columns={'Order_Demand':'Total_Neg'})

# Extract positve values and aggregate by product
positive = df.loc[df['Order_Demand'] > 0]
positive_pv = pd.pivot_table(positive, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum)

# Add a column with corresponding total positive value by product and calculate percentage, sort = desc
negative_pv['Total_Pos'] = positive_pv.loc[positive_pv.index.isin(negative_pv.index),]
negative_pv['Percentage'] = abs(negative_pv['Total_Neg'])*100/negative_pv['Total_Pos']
negative_pv = negative_pv.sort_values('Percentage', ascending = False)

print("Percentage ranges from",negative_pv['Percentage'].min(),"to",negative_pv['Percentage'].max())

In [ ]:
print("Demand of Product_0319")
print(df.loc[df['Product_Code'] == 'Product_0319'].head(10))
print("\t")
print("Demand of Product_0568")
print(df.loc[df['Product_Code'] == 'Product_0568'].head(10))

<p style="color:MediumVioletRed;font-size:20px;">Missing Values</p>

In [ ]:
print("The number of products with null is",len(df['Product_Code'].value_counts().index))

<p style="color:MediumVioletRed;font-size:20px;">Aggregate data by month</p>
Forecasting is performed at monthly horizons, thus the dataset should first be aggregated by month. Date is extracted with Month & Year only.

In [ ]:
df['Date'] = df['Date'].dt.to_period('M')
df = df.rename(columns = {"Date": 'Period'})
df = df.groupby(['Product_Code','Period'])['Order_Demand'].sum().reset_index().sort_values('Period'
            ).reset_index().drop('index',axis=1)
df.head()

In [ ]:
full_period = pd.date_range('2011-01-01','2016-12-31', freq='MS').to_period('M')
full_period = set(full_period)
data_period = set(df['Period'])
full_period.difference(data_period)
# The missing periods are 5 months in 2011, including Feb, Mar, Apr, Jul, and Aug.
# There are various possible reasons for the missing periods: No demands are in these months, warehouses to be
# closed in these months for some reason, missing data in these periods, etc.
# To ensure that the training data will not be misleading, all data before Sep 2011 will be removed.
data = data.loc[df['Period'] > '2011-08']

Check to see which products are eligible for forecasting

In [ ]:
df = df.loc[df['Period']<'2017-01']

In [ ]:
# Criteria 2: Remove stopped products
latest_datamonth = data.groupby('Product_Code')['Period'].max().reset_index()
latest_datamonth = latest_datamonth.loc[latest_datamonth['Period'] > '2015-12']
data = df.loc[df['Product_Code'].isin(latest_datamonth['Product_Code'])]

In [ ]:
import pandas as pd
from operator import attrgetter


# Group by 'Product_Code' and calculate the min and max of 'Period'
duration_data = data.groupby('Product_Code').agg({'Period': ['min', 'max']}).reset_index()

# Calculate Duration as the difference between max and min periods, then add 1
duration_data['Duration'] = (duration_data[('Period', 'max')] - duration_data[('Period', 'min')]).apply(attrgetter('n')) + 1

# Filter the rows where Duration is greater than 24
duration_data = duration_data.loc[duration_data['Duration'] > 24]

# Filter the original data to keep only the relevant 'Product_Code'
data = data.loc[data['Product_Code'].isin(duration_data['Product_Code'])]

# Display the results
print(duration_data)
print(data)


Construct time series in a columnar format

In [ ]:
data = pd.pivot_table(data, values = 'Order_Demand', index = 'Period', columns = 'Product_Code',aggfunc=np.sum
                     ).reset_index().rename_axis("", axis="columns")

#Fill in missing values with 0. Months with missing values are implied to have zero demands.
data = data.fillna(0)
data = data.set_index('Period')
data.head()

In [ ]:
data=pd.read_csv("Historical Product Demand.csv")
fig = go.Figure(data=[go.Pie(labels=["Product_0001", "Product_0002"], values=data['Order_Demand'].value_counts(), pull=[0, 0.2])])
fig.update_layout(title_text="Historical product demand", plot_bgcolor=colors[3], paper_bgcolor=colors[4], font_color=colors[5])
fig.update_traces(marker=dict(colors=[colors[1], colors[0]]))
fig.show()

In [ ]:
# Group data based on 'Date' and calculate total 'Order_Demand'
order_demand_per_date = data.head().groupby('Date').sum()['Order_Demand']

# Group data based on 'Date' and 'Product_Category' or 'Warehouse' to count orders
category_demand = data.head().groupby(['Date', 'Product_Category']).sum()['Order_Demand']
warehouse_demand = data.head().groupby(['Date', 'Warehouse']).sum()['Order_Demand']

# Prepare DataFrame for plotting
category_demand_df = category_demand.unstack().fillna(0)
warehouse_demand_df = warehouse_demand.unstack().fillna(0)

# Subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=(
        "Order Demand by Category", 
        "Order Demand by Warehouse", 
        "Total Order Demand Over Time"
    )
)

# Plot: Demand by Category
for col in category_demand_df.columns:
    fig.add_trace(
        go.Scatter(x=category_demand_df.index, y=category_demand_df[col], name=f"Category: {col}"),
        row=1, col=1
    )

# Plot: Demand by Warehouse
for col in warehouse_demand_df.columns:
    fig.add_trace(
        go.Scatter(x=warehouse_demand_df.index, y=warehouse_demand_df[col], name=f"Warehouse: {col}"),
        row=1, col=2
    )

# Plot: Total Order Demand Over Time
fig.add_trace(
    go.Scatter(x=order_demand_per_date.index, y=order_demand_per_date, name="Total Demand", marker={'color': colors[2]}),
    row=2, col=1
)

# Layout and axes configs
fig.update_layout(
    showlegend=True,
    title_text="Order Demand Analysis Over Time",
    height=650,
    plot_bgcolor=colors[3],
    paper_bgcolor=colors[4],
    font_color=colors[5]
)
fig.update_xaxes(gridcolor=colors[6], color=colors[5])
fig.update_yaxes(gridcolor=colors[6], color=colors[5])

# Axes labels
fig['layout']['xaxis']['title'] = 'Date'
fig['layout']['yaxis']['title'] = 'Order Demand'
fig['layout']['xaxis2']['title'] = 'Date'
fig['layout']['yaxis2']['title'] = 'Order Demand'
fig['layout']['xaxis3']['title'] = 'Date'
fig['layout']['yaxis3']['title'] = 'Order Demand (Total)'

fig.show()


In [ ]:
# Layout and axes configuration with a black background
fig.update_layout(
    title="Max-Mean-Min Graph for Order Demand by Product Category",
    title_x=0.5,
    height=600,
    plot_bgcolor="black",  # Set plot background to black
    paper_bgcolor="black",  # Set paper background to black
    font_color="white",  # Set font color to white for visibility
    xaxis_title="Product Category",
    yaxis_title="Order Demand"
)
fig.update_xaxes(gridcolor="gray", zerolinecolor="gray", color="white")  # Set axes grid and text to light gray/white
fig.update_yaxes(gridcolor="gray", zerolinecolor="gray", color="white")


In [ ]:
# Calculate rows and columns dynamically based on the number of categories
num_categories = len(categories)
cols = 6  # Fixed number of columns
rows = (num_categories // cols) + (num_categories % cols > 0)  # Calculate rows dynamically

# Initialize figure
fig = plt.figure(figsize=(22, 14))
fig.tight_layout(pad=3.0)

# Plot parameters
plt.rcParams["figure.autolayout"] = True
plt.rcParams["axes.facecolor"] = "black"  # Set axes background to black
plt.rcParams["axes.linewidth"] = 3
plt.rcParams["xtick.color"] = "white"
plt.rcParams["ytick.color"] = "white"
plt.rcParams["axes.labelcolor"] = "white"
plt.rcParams["text.color"] = "white"
plt.rcParams["figure.titlesize"] = 18
fig.patch.set_facecolor("black")  # Set figure background to black
plt.suptitle("Product Demand Density Plot by Category", color="white")

# Subplots for each Product_Category
for idx, category in enumerate(categories):
    plt.subplot(rows, cols, idx + 1)  # Adjust grid size dynamically
    
    # Select data for the current category
    category_data = data_grouped.get_group(category)
    
    # Plot KDE for Order_Demand
    sns.kdeplot(
        x=category_data['Order_Demand'], 
        linewidth=3, 
        color="cyan", 
        label="Order Demand"
    )
    
    # Customize subplot
    plt.title(f"Category: {category}", color="white")
    plt.xlabel("Order Demand", color="white")
    plt.ylabel("Density", color="white")
    plt.legend(labels=['Order Demand'], loc='upper right', frameon=False)

# Display the plot
plt.show()


In [ ]:
pip install pulp

# Optimization Techniques

<p style="color:MediumVioletRed;font-size:20px;">Linear Programming</p>

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, LpInteger

# Linear Programming (LP) Example: Resource Allocation
def linear_programming_example():
    # Create a Linear Programming problem
    lp_problem = LpProblem("LP_Resource_Allocation", LpMaximize)

    # Decision variables (continuous)
    x = LpVariable("x", lowBound=0)  # Product A
    y = LpVariable("y", lowBound=0)  # Product B

    # Objective Function: Maximize profit
    lp_problem += 40 * x + 30 * y, "Total_Profit"

    # Constraints
    lp_problem += 2 * x + y <= 50, "Material_Availability"
    lp_problem += x + 2 * y <= 40, "Labor_Availability"

    # Solve the problem
    lp_problem.solve()
    print("Linear Programming Solution:")
    for variable in [x, y]:
        print(f"{variable.name} = {variable.varValue}")
    print(f"Total Profit = {lp_problem.objective.value()}")




# Run all examples
linear_programming_example()



<p style="color:MediumVioletRed;font-size:20px;">Integer Programming</p>

In [ ]:
# Integer Programming (IP) Example: Scheduling
def integer_programming_example():
    # Create an Integer Programming problem
    ip_problem = LpProblem("IP_Scheduling", LpMaximize)

    # Decision variables (integer)
    x = LpVariable("x", lowBound=0, cat=LpInteger)  # Task 1
    y = LpVariable("y", lowBound=0, cat=LpInteger)  # Task 2

    # Objective Function: Maximize task completion value
    ip_problem += 10 * x + 15 * y, "Total_Value"

    # Constraints
    ip_problem += x + y <= 5, "Max_Allowed_Tasks"
    ip_problem += 2 * x + 3 * y <= 10, "Resource_Constraint"

    # Solve the problem
    ip_problem.solve()
    print("\nInteger Programming Solution:")
    for variable in [x, y]:
        print(f"{variable.name} = {variable.varValue}")
    print(f"Total Value = {ip_problem.objective.value()}")
    
    
integer_programming_example()

<p style="color:MediumVioletRed;font-size:20px;">Integer Programming</p>

In [ ]:
# Mixed-Integer Linear Programming (MILP) Example: Energy Optimization
def mixed_integer_linear_programming_example():
    # Create a MILP problem
    milp_problem = LpProblem("MILP_Energy_Optimization", LpMaximize)

    # Decision variables (mixed: continuous and integer)
    x = LpVariable("x", lowBound=0)  # Continuous variable
    y = LpVariable("y", lowBound=0, cat=LpInteger)  # Integer variable

    # Objective Function: Maximize efficiency
    milp_problem += 25 * x + 40 * y, "Efficiency"

    # Constraints
    milp_problem += x + y <= 10, "Total_Limit"
    milp_problem += 3 * x + 2 * y <= 18, "Energy_Constraint"

    # Solve the problem
    milp_problem.solve()
    print("\nMixed-Integer Linear Programming Solution:")
    for variable in [x, y]:
        print(f"{variable.name} = {variable.varValue}")
    print(f"Efficiency = {milp_problem.objective.value()}")
    
    
mixed_integer_linear_programming_example()



# Evalution Metrics

In [ ]:
# Function to calculate cost, revenue, profit, and profit margin
def evaluate_metrics(cost, revenue):
    # Calculate profit
    profit = revenue - cost
    
    # Calculate profit margin
    if revenue != 0:  # Avoid division by zero
        profit_margin = (profit / revenue) * 100
    else:
        profit_margin = 0
    
    # Display the results
    print(f"Cost: ${cost}")
    print(f"Revenue: ${revenue}")
    print(f"Profit: ${profit}")
    print(f"Profit Margin: {profit_margin:.2f}%")

# Example usage
cost = float(input("Enter the total cost: $"))
revenue = float(input("Enter the total revenue: $"))

evaluate_metrics(cost, revenue)
